<a href="https://colab.research.google.com/github/AndrewJGroves/PlottingInformation/blob/main/NuclearMissle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plotting Nuclear Missle Data

Code to take data on nuclear missiles by country and plot them on a graph

In [1]:
pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11686 sha256=ea326cec45838da096ac0598ffbbfe51a38d26f6763ea9d30680acdd50c403f0
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


Start my taking the information from wikipedia and cleaning it

In [62]:
import pandas as pd
import wikipedia as wp
import numpy as np

html = wp.page("Historical nuclear weapons stockpiles and nuclear tests by country").html().encode("UTF-8")
df = pd.read_html(html)[0] 

#Finding the countires and putting them into the index of a new data frame
countries=list(df.columns[0]) 
countries=countries[1:-1]
new_df=pd.DataFrame(index=countries)

for i in range(1,len(df.columns)-1):
  li=list(df.columns[i])
  year=li[0]
  li=li[1:-1]
  new_li=[]
  for i in li: #Cleaning the data and making the numbers useable
    if i == 'â\x80\x94':
      new_li.append(0)
    else:
      x=i.split('-')
      if len(x) > 1:
        x=x[1]
      else:
        x=x[0]
      x=x.split('[',1)
      x=x[0]
      x=x.split()
      x=x[0]
      x=x.replace(',','')
      try:
        x=int(x)
      except:
        x=np.nan
      new_li.append(x)
  new_df[year]=new_li
new_df.fillna(81,inplace=True) #Single value was hard to deal with and fast to input by hand due to the size of the dataframe
new_df.head(20)


,1945,1950,1955,1960,1965,1970,1975,1980,1985,1990,1995,2000,2005,2014,2020
United States,2,299,2422,18638,31149,26008,27519,23368,21392,10904.0,10577,8360,7700,7260,5800
Soviet Union Russia,0,5,200,1605,6129,11643,19055,30062,39197,37000.0,27000,21500,17000,7500,6375
China,0,0,0,0,5,75,180,205,243,232.0,234,232,235,260,320
France,0,0,0,0,32,36,188,250,360,505.0,500,470,350,300,290
United Kingdom,0,0,14,42,436,394,492,492,422,422.0,422,281,281,215,215
India,0,0,0,0,0,0,0,1,3,7.0,14,28,44,110,150
Pakistan,0,0,0,0,0,0,0,0,0,4.0,13,28,38,120,160
Israel,0,0,0,0,0,8,20,31,42,53.0,63,72,80,80,90
North Korea,0,0,0,0,0,0,0,0,0,1.0,2,2,8,8,40
South Africa,0,0,0,0,0,0,0,0,3,6.0,0,0,0,0,0


Now we have a dataframe to work with we can start to do plotting

In [ ]:
import seaborn as sns
print(len(new_df.columns))
print(len(new_df.index))
sns.lineplot(data=new_df,x=new_df.columns)